In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_1')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_1


In [3]:
!python --version

Python 3.7.10


In [4]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

In [5]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [6]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [7]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [8]:
df_test

,index,class,title
0,1800,體育,如果騎士火箭進入總決賽，誰的勝算大？
1,1801,體育,從個人競技狀態來看，三個階段的詹姆斯，哪個最強？
2,1802,體育,騎士總冠軍！地球人誰能阻擋詹姆斯？史上最佳就是他！打服所有人
3,1803,體育,詹姆斯絕殺，騎士3比0，猛龍懷疑人生
4,1804,體育,騎士和步行者戰成搶七險勝，而猛龍即將被橫掃，步行者跟猛龍的區別在哪裡？
...,...,...,...
595,2395,遊戲,如何玩好《陰陽師》？
596,2396,遊戲,網賺——玩遊戲網賺也不錯呦
597,2397,遊戲,迷你世界怎麼跳四段跳？
598,2398,遊戲,DNF5000罐子挑戰遠古遺願，玩家最後只說了兩個字「坑爹」


In [9]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [10]:
train_titles[0] ##test_titles : {0: '亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列',1: , ...}

'亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列'

In [11]:
train_classes[0]  ##test_classes : {0: '體育', 1:  }

'體育'

In [12]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞(WS) + POS

In [13]:
ws = WS('../ckip_data/')
pos = POS('../ckip_data/')

In [14]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws(
      [title]#,
      #sentence_segmentation = True, # To consider delimiters
      #segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
      ) # This is the defualt set of delimiters
    word_p = pos(word_s)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [15]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws(
      [title]#,
      #sentence_segmentation = True, # To consider delimiters
      #segment_delimiter_set = {",", "。", ":", "?", "!", ";"}
      ) # This is the defualt set of delimiters
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [16]:
train_title_cuts[0] #0~1799 {0: [('亞洲杯', 'Nb'),  ('奪', 'VC'),  ('冠賠率', 'Na'),  ('：', 'COLONCATEGORY'),  ('日本', 'Nc'),] ,1: [..]

[('亞洲杯', 'Nb'),
 ('奪', 'VC'),
 ('冠賠率', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日本', 'Nc'),
 ('、', 'PAUSECATEGORY'),
 ('伊朗', 'Nc'),
 ('領銜', 'VB'),
 (' ', 'WHITESPACE'),
 ('中國', 'Nc'),
 ('竟', 'D'),
 ('與', 'P'),
 ('泰國', 'Nc'),
 ('並列', 'VG')]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [17]:
test_title_cuts[1800] #1800~2399

[('如果', 'Cbb'),
 ('騎士', 'Na'),
 ('火箭', 'Na'),
 ('進入', 'VCL'),
 ('總決賽', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('誰', 'Nh'),
 ('的', 'DE'),
 ('勝算', 'Na'),
 ('大', 'VH'),
 ('？', 'QUESTIONCATEGORY')]

# Bag of Words (BOW)

In [18]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1
# END YOUR CODE

In [19]:
word2index['溫暖']

1520

In [20]:
index2word[1520]

'溫暖'

In [21]:
def get_bow_vector(pairs, word2index): ##pairs: (ws,pos)
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index:
            vector[word2index[word]] += 1
    # END YOUR CODE
    return vector

In [22]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [23]:
pos_analysis = {}
for _, pairs in train_title_cuts.items(): ### index,ws,pos
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['泰拳王西', '馮天睿', '韋德', '碧生源', '昌圖', '切沃', '張大仙', '桑德羅', '勇士隊', '楊柳夏', '遼足', '游古', '王楚欽', '施泰納', 'J羅', '易', '雀巢', '石川佳純', '萊科寧', '榮威', '·龜茲', '林美景', '安卓', '劉偉', '朗多', '台積電', '建聯', '李嘉誠', '里坦克', '盼保', '德羅巴', '加威廉', '科爾', '布茲德里克', '德羅贊', '東決', '劉集', '白起', '韓信', '韓寒', '埃姆雷', '劉德華', '網傳', '王昭君', '朱嘯虎', '詹', '蓋倫', '騎士隊', '科斯塔', '盧克肖', '男乒', '德甲', '烏迪內斯', '谷歌', '李大霄', '拉卡拉', '藍星', '朱之文', '劉代全', '雅桑克萊', '泰山', '伯德', '美年達', '巴爾韋德', '章澤天', '中超', '張藝謀', 'G7', '王者榮耀', '德羅', '瓜迪奧拉', 'U23', '希金斯', '每日', '阿迪薩亞', '高傭', '波爾', '凱爾特', '泰倫', '海爾', '精准', '東皇', '凱西', '納米 華', '亞洲杯', '堪比比薩', '秦緒文', '余則成', '艷武漢', '溫格', '羅永浩', '卡帥', '弗格森', '安迪爾森', '馬克', '星巴克', '泰倫盧', '阿里雲', 'G3，76', '溫氏']
VC : ['爭', '募', '共進', '生產', '登頂', '指責', '放大', '花', '拼', '誤觸', '食用', '購買', '火遍', '好', '參', '包裝', '噴', '匯總', '解說', '利用', '推薦', '鋪開', '重建', '高產', '培育', '觀看', '提高', '舔', '戲耍', '開好', '打死', '學到', '拍拍', '過上', '扔上', '扔丟', '拍完', '提升到', '發售', '做好', '賒', '隱藏', '振興', '經營', '打開', '迎來', '綁定', '結出', '找', '分銷', '進行', '眾籌', '洩露', '接待', '買',

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [24]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
        'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
        'Caa', 'Cab', 'Cba', 'Cbb',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [25]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    similarity = np.sum(bow1 * bow2) / (len_bow1 * len_bow2)
    # END YOUR CODE
    return similarity

In [26]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [27]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [28]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [29]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333]),
 '體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ])}

# Group mean vector: 測試

In [30]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [31]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})
predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})
predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})


predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})

predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})

predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})

predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})

predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})

predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})
